In [209]:
import pandas as pd
import numpy as np
import nltk

from google.colab import files
uploaded = files.upload()

Saving atis_intents_train.csv to atis_intents_train.csv


In [210]:
df = pd.read_csv('/content/atis_intents_train.csv')
df.head()

,X,Y,Class
0,atis_flight,i want to fly from boston at 838 am and arriv...,NaN
1,atis_flight,what flights are available from pittsburgh to...,NaN
2,atis_flight_time,what is the arrival time in san francisco for...,NaN
3,atis_airfare,cheapest airfare from tacoma to orlando,NaN
4,atis_airfare,round trip fares from pittsburgh to philadelp...,NaN


In [211]:
classes = {
    "atis_flight": 0,
    "atis_airline": 1,
    "atis_airfare": 2,
    "atis_ground_service": 3,
    "atis_abbreviation": 4,
    "atis_city": 5,
    "atis_aircraft": 6,
    "atis_quantity": 7,
    "atis_capacity": 8,
    "atis_flight_time": 9,
}

# Mapping the classes
df['Class'] = df['X'].map(classes)
df = df.dropna(subset=['Class']).astype({'Class': 'int64'})
df['Class'].dtype
df.shape

(4834, 3)

In [212]:
sentences = df['Y']

In [214]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [215]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

sentences = sentences.apply(lambda text: remove_stopwords(text.lower()))
sentences.head()

0       want fly boston 838 arrive denver 1110 morning
1    flights available pittsburgh baltimore thursda...
2    arrival time san francisco 755 flight leaving ...
3                      cheapest airfare tacoma orlando
4    round trip fares pittsburgh philadelphia 1000 ...
Name: Y, dtype: object

In [216]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

sentences = sentences.apply(lambda text: lemmatize_words(text))
sentences.head()

0       want fly boston 838 arrive denver 1110 morning
1    flight available pittsburgh baltimore thursday...
2    arrival time san francisco 755 flight leaving ...
3                      cheapest airfare tacoma orlando
4    round trip fare pittsburgh philadelphia 1000 d...
Name: Y, dtype: object

In [217]:
bow = {}
i = 1
for row in sentences:
  for word in row.split(" "):
    if word not in bow:
      bow[word] = i
      i += 1

In [218]:
X = []
for i in sentences:
  X.append(nltk.word_tokenize(i))

In [219]:
X

[['want', 'fly', 'boston', '838', 'arrive', 'denver', '1110', 'morning'],
 ['flight', 'available', 'pittsburgh', 'baltimore', 'thursday', 'morning'],
 ['arrival',
  'time',
  'san',
  'francisco',
  '755',
  'flight',
  'leaving',
  'washington'],
 ['cheapest', 'airfare', 'tacoma', 'orlando'],
 ['round', 'trip', 'fare', 'pittsburgh', 'philadelphia', '1000', 'dollar'],
 ['need', 'flight', 'tomorrow', 'columbus', 'minneapolis'],
 ['kind', 'aircraft', 'used', 'flight', 'cleveland', 'dallas'],
 ['show', 'flight', 'pittsburgh', 'los', 'angeles', 'thursday'],
 ['flight', 'boston', 'washington'],
 ['kind', 'ground', 'transportation', 'available', 'denver'],
 ['show', 'flight', 'dallas', 'san', 'francisco'],
 ['show', 'flight', 'san', 'diego', 'newark', 'way', 'houston'],
 ['cheapest', 'flight', 'boston', 'bwi'],
 ['flight', 'baltimore', '6', 'pm'],
 ['show', 'first', 'class', 'fare', 'boston', 'denver'],
 ['show', 'ground', 'transportation', 'denver'],
 ['flight', 'denver', 'pittsburgh', 'lea

In [220]:
bow

{'want': 1,
 'fly': 2,
 'boston': 3,
 '838': 4,
 'arrive': 5,
 'denver': 6,
 '1110': 7,
 'morning': 8,
 'flight': 9,
 'available': 10,
 'pittsburgh': 11,
 'baltimore': 12,
 'thursday': 13,
 'arrival': 14,
 'time': 15,
 'san': 16,
 'francisco': 17,
 '755': 18,
 'leaving': 19,
 'washington': 20,
 'cheapest': 21,
 'airfare': 22,
 'tacoma': 23,
 'orlando': 24,
 'round': 25,
 'trip': 26,
 'fare': 27,
 'philadelphia': 28,
 '1000': 29,
 'dollar': 30,
 'need': 31,
 'tomorrow': 32,
 'columbus': 33,
 'minneapolis': 34,
 'kind': 35,
 'aircraft': 36,
 'used': 37,
 'cleveland': 38,
 'dallas': 39,
 'show': 40,
 'los': 41,
 'angeles': 42,
 'ground': 43,
 'transportation': 44,
 'diego': 45,
 'newark': 46,
 'way': 47,
 'houston': 48,
 'bwi': 49,
 '6': 50,
 'pm': 51,
 'first': 52,
 'class': 53,
 '7': 54,
 'information': 55,
 'tuesday': 56,
 'please': 57,
 'give': 58,
 'next': 59,
 'week': 60,
 'would': 61,
 'like': 62,
 'united': 63,
 'airline': 64,
 'list': 65,
 'plane': 66,
 'american': 67,
 "i'd": 68

In [221]:
import pickle

In [222]:
with open('bow.pickle', 'wb') as file:
  pickle.dump(bow, file, protocol=pickle.HIGHEST_PROTOCOL)

In [223]:
labels = df['Class']

labels

0       0
1       0
2       9
3       2
4       2
       ..
4829    2
4830    0
4831    1
4832    0
4833    0
Name: Class, Length: 4834, dtype: int64

In [224]:
sentences

0          want fly boston 838 arrive denver 1110 morning
1       flight available pittsburgh baltimore thursday...
2       arrival time san francisco 755 flight leaving ...
3                         cheapest airfare tacoma orlando
4       round trip fare pittsburgh philadelphia 1000 d...
                              ...                        
4829       airfare flight denver pittsburgh delta airline
4830                   flight denver baltimore via dallas
4831                                   airline fly denver
4832     continental fly boston san francisco stop denver
4833                    delta flight denver san francisco
Name: Y, Length: 4834, dtype: object

In [225]:
trainData = []
for i in X:
  temp = []
  for word in i:
    if word in bow:
      a = bow.get(word)
      temp.append(a)
  trainData.append(temp)

In [226]:
trainData

[[1, 2, 3, 4, 5, 6, 7, 8],
 [9, 10, 11, 12, 13, 8],
 [14, 15, 16, 17, 18, 9, 19, 20],
 [21, 22, 23, 24],
 [25, 26, 27, 11, 28, 29, 30],
 [31, 9, 32, 33, 34],
 [35, 36, 37, 9, 38, 39],
 [40, 9, 11, 41, 42, 13],
 [9, 3, 20],
 [35, 43, 44, 10, 6],
 [40, 9, 39, 16, 17],
 [40, 9, 16, 45, 46, 47, 48],
 [21, 9, 3, 49],
 [9, 12, 50, 51],
 [40, 52, 53, 27, 3, 6],
 [40, 43, 44, 6],
 [9, 6, 11, 19, 50, 51, 54, 51],
 [31, 55, 9, 56, 19, 12, 39, 39, 3, 3, 12],
 [57, 58, 9, 3, 11, 13, 59, 60],
 [61, 62, 2, 6, 11, 63, 64],
 [40, 9, 16, 45, 46],
 [57, 65, 52, 53, 9, 63, 6, 12],
 [35, 66, 37, 67, 64],
 [62, 55, 69, 6, 11, 70],
 [62, 71, 9, 70, 6],
 [64, 72, 6, 11, 70],
 [40, 9, 3, 11, 73, 59, 60, 74, 3, 75, 76, 51],
 [70, 43, 44],
 [77, 31, 78, 39, 3, 79, 80],
 [40, 21, 25, 26, 27, 12, 39],
 [40, 9, 81, 82, 83, 84, 64, 85, 86],
 [9, 81, 82, 87],
 [57, 88, 9, 89, 28, 16, 17, 58, 9, 90],
 [31, 9, 39, 16, 17],
 [27, 9, 11, 28],
 [40, 64, 52, 53, 9],
 [9, 91, 92, 93, 94, 95],
 [62, 96, 9, 11, 70],
 [40, 9,

In [228]:
import tensorflow as tf

padded_data = tf.keras.preprocessing.sequence.pad_sequences(trainData, maxlen=16, padding='post')
padded_data

array([[  1,   2,   3, ...,   0,   0,   0],
       [  9,  10,  11, ...,   0,   0,   0],
       [ 14,  15,  16, ...,   0,   0,   0],
       ...,
       [ 64,   2,   6, ...,   0,   0,   0],
       [244,   2,   3, ...,   0,   0,   0],
       [ 89,   9,   6, ...,   0,   0,   0]], dtype=int32)

In [229]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(1, 110))
scale_data = scaler.fit_transform(padded_data)
scale_data.shape

(4834, 16)

In [230]:
from sklearn.model_selection import train_test_split

Xtr, Xte, Ytr, Yte = train_test_split(scale_data, labels, test_size = 0.2)

In [231]:
Xtr.shape
# Ytr.shape

(3867, 16)

In [232]:
Ytr.shape

(3867,)

In [233]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Flatten, Embedding, Dropout, Input, SimpleRNN
from tensorflow.keras.optimizers import Adam, SGD

In [234]:
model = Sequential([
              LSTM(128, activation="tanh", return_sequences=True),
              LSTM(64, activation="tanh", return_sequences=True),
              LSTM(32, activation="tanh"),
              Dropout(0.2),
              Flatten(),
              Dense(64,activation="relu"),
              Dropout(0.2),
              Dense(32, activation="relu"),
              Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.01), metrics=['accuracy'])

Xtr = np.expand_dims(Xtr, axis=-1)

print(Xtr.shape)


(3867, 16, 1)


In [235]:
hist = model.fit(Xtr, Ytr, epochs = 50)

Epoch 1/50
121/121 [==============================] - 18s 78ms/step - loss: 1.1532 - accuracy: 0.7533
Epoch 2/50
121/121 [==============================] - 8s 66ms/step - loss: 0.9756 - accuracy: 0.7634
Epoch 3/50
121/121 [==============================] - 8s 69ms/step - loss: 0.9636 - accuracy: 0.7634
Epoch 4/50
121/121 [==============================] - 8s 68ms/step - loss: 0.9222 - accuracy: 0.7675
Epoch 5/50
121/121 [==============================] - 7s 57ms/step - loss: 0.9080 - accuracy: 0.7732
Epoch 6/50
121/121 [==============================] - 8s 67ms/step - loss: 0.8901 - accuracy: 0.7717
Epoch 7/50
121/121 [==============================] - 7s 57ms/step - loss: 0.8793 - accuracy: 0.7719
Epoch 8/50
121/121 [==============================] - 8s 67ms/step - loss: 0.8277 - accuracy: 0.7753
Epoch 9/50
121/121 [==============================] - 8s 65ms/step - loss: 0.8068 - accuracy: 0.7748
Epoch 10/50
121/121 [==============================] - 7s 58ms/step - loss: 0.7530 - accur

In [176]:
model.evaluate(Xte, Yte)
model.save('intentClassification.h5')

31/31 [==============================] - 2s 19ms/step - loss: 0.5454 - accuracy: 0.8460


In [197]:
import random
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from keras.models import load_model

lemmatizer = WordNetLemmatizer()

In [205]:
words = pickle.load(open('bow.pickle', 'rb'))

def generate_response(sentence):
    greetings_responses = ['Hi!', 'Hey, How can I help you today', 'Hello I am AirBot to help you out', 'Hey there!']
    sentence = str(sentence).lower()
    if sentence == "hi" or sentence == "hello" or sentence == "Hello" or sentence == "Hi":
        return random.choice(greetings_responses)
    elif sentence == 'bye':
        return "Bye"
    else:
        c = preprocess_text(sentence)
        return c

def predict_model(sentence):
  classes = ["atis_flight", "atis_airline", "atis_airfare", "atis_ground_service", "atis_abbreviation","atis_city","atis_aircraft","atis_quantity","atis_capacity","atis_flight_time"]
  model_path = "/content/intentClassification.h5"
  model = tf.keras.models.load_model(model_path)
  output = model.predict(np.expand_dims(sentence, axis=0))
  c = np.argmax(output)
  print(classes[c])
  return classes[c]


def preprocess_text(sentence):
    with open("/content/bow.pickle", "rb") as file:
        bow = pickle.load(file)

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(sentence)
    preprocessed_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    vectored_text = []
    for i in preprocessed_sentence:
        if i in bow:
            a = bow.get(i)
            vectored_text.append(a)
        else:
            vectored_text.append(0)

    # vectored_text = vectored_text.append(0*(16-len(vectored_text)))
    # vectored_text = tf.keras.preprocessing.sequence.pad_sequences(vectored_text, padding='post', maxlen=16)

    for i in range(len(vectored_text)-1, 15):
        vectored_text.append(0)
    vectored_text = np.array(vectored_text)

    # print(vectored_text)
    # print(vectored_text.shape)

    c = predict_model(vectored_text)
    return c

1/1 [==============================] - 2s 2s/step
atis_ground_service
{'text': 'Let me find you ground services!', 'intent': 'atis_ground_service'}
